## Images to train, test

In [9]:
import os

TRAIN_SPLIT = 0.8

RAW_IMAGES = './hard_hat/images/'
RAW_ANNOTATIONS = './hard_hat/annotations/'
TRAIN_DIR = './data/train/'
TEST_DIR = './data/test/'

ANNOTATIONS_DIR = './annotations/'

In [4]:
# image_files = os.listdir(RAW_IMAGES)
# for idx, file in enumerate(image_files[:500]):
#     annot = file.split('.')[0] + '.xml'

#     try:
#         if idx < len(image_files[:500])*TRAIN_SPLIT:
#             !cp {RAW_ANNOTATIONS + annot} {TRAIN_DIR}
#             !cp {RAW_IMAGES + file} {TRAIN_DIR}
#         else:
#             !cp {RAW_ANNOTATIONS + annot} {TEST_DIR}
#             !cp {RAW_IMAGES + file} {TEST_DIR}
#     except:
#         print('{} ERROR!'.format(file))

## Label map and create .pbtxt

In [2]:
labels = [{'name': 'helmet', 'id': 1}, {'name': 'head', 'id': 2}, {'name': 'person', 'id': 3}]
labels

[{'name': 'helmet', 'id': 1},
 {'name': 'head', 'id': 2},
 {'name': 'person', 'id': 3}]

In [7]:
# with open(ANNOTATIONS_DIR + 'label_map.pbtxt', 'w') as f:
#     for label in labels:
#         f.write('item{\n')
#         f.write('\tname:\'{}\'\n'.format(label['name']))
#         f.write('\tid:{}\n'.format(label['id']))
#         f.write('}\n')

## .xml to .record

In [ ]:
import glob
import pandas as pd
import io
import xml.etree.ElementTree as ET
import tensorflow as tf
from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from collections import namedtuple

label_map = label_map_util.load_labelmap(LABEL_MAP_FILE)
label_map_dict = label_map_util.get_label_map_dict(label_map)

tf.compat.v1.disable_eager_execution() 
tf.compat.v1.reset_default_graph()
png_img_pl = tf.compat.v1.placeholder(tf.string)
png_enc = tf.image.decode_png(png_img_pl, channels = 3)
    # Set how much quality of image you would like to retain while conversion
png_to_jpeg = tf.image.encode_jpeg(png_enc, format = 'rgb', quality = 100)

def xml_to_csv(path):

    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                         int(root.find('size').find('width').text),
                         int(root.find('size').find('height').text),
                         member[0].text,
                         int(member.find("bndbox").find('xmin').text),
                         int(member.find("bndbox").find('ymin').text),
                         int(member.find("bndbox").find('xmax').text),
                         int(member.find("bndbox").find('ymax').text)
                        )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def class_text_to_int(row_label):
    return label_map_dict[row_label]


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def is_png_image(filename):
    ext = os.path.splitext(filename)[1].lower()
    return ext == '.png'

def convert_png_to_jpeg(img):
    with tf.compat.v1.Session() as sess:    
        return sess.run(png_to_jpeg, feed_dict = {png_img_pl: img})


def create_tfrec_file(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        image_data = fid.read()
        
        if is_png_image(os.path.join(path, '{}'.format(group.filename))): # If files are .png - format convert them on the fly
            image_data = convert_png_to_jpeg(image_data)

        encoded_jpg = image_data    
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def create_tfrecord(OUTPUT,INPUT_IMAGE,INPUT_XML):        
    writer = tf.io.TFRecordWriter(OUTPUT)
    path = os.path.join(INPUT_IMAGE)
    examples = xml_to_csv(INPUT_XML)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_ref_file = create_tfrec_file(group, path)
        writer.write(tf_ref_file.SerializeToString())        
    print('Successfully created the TFRecord file: {}'.format(OUTPUT))
    writer.close()

In [5]:
!python generate_tfrecord_from_xml.py -x {TRAIN_DIR} -i {TRAIN_DIR} -l {ANNOTATIONS_DIR + 'label_map.pbtxt'} -o {ANNOTATIONS_DIR + 'train.record'}
!python generate_tfrecord_from_xml.py -x {TEST_DIR} -i {TEST_DIR} -l {ANNOTATIONS_DIR + 'label_map.pbtxt'} -o {ANNOTATIONS_DIR + 'test.record'}

Successfully created the TFRecord file: ./annotations/train.record
Successfully created the TFRecord file: ./annotations/test.record


## Copy config to custom model dir

In [10]:
# PRETRAINED_DIR = './pretrained_models/centernet_mobilenetv2_fpn_od/'
# MY_MODEL_DIR = './models/my_centernet/'

PRETRAINED_DIR = './pretrained_models/efficientdet_d0_coco17_tpu-32/'
MY_MODEL_DIR = './models/my_effiecientdet/'

In [6]:
!mkdir {MY_MODEL_DIR}
!cp {PRETRAINED_DIR + 'pipeline.config'} {MY_MODEL_DIR}

mkdir: cannot create directory ‘./models/my_effiecientdet/’: File exists


## Update config for transfer learning

In [6]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [8]:
config = config_util.get_configs_from_pipeline_file(MY_MODEL_DIR + 'pipeline.config')
# config

In [9]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(MY_MODEL_DIR + 'pipeline.config', 'r') as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [10]:
pipeline_config.model.ssd.num_classes = 3
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_DIR + 'checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = 'detection'
pipeline_config.train_input_reader.label_map_path = ANNOTATIONS_DIR + 'label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATIONS_DIR + 'train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATIONS_DIR + 'label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATIONS_DIR + 'test.record']

In [11]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(MY_MODEL_DIR + 'pipeline.config', 'wb') as f:
    f.write(config_text)

## Train (terminal)

In [21]:
# !echo python ../models/research/object_detection/model_main_tf2.py --model_dir={MY_MODEL_DIR} --pipeline_config_path={MY_MODEL_DIR + 'pipeline.config'} --num_train_steps=5000
!echo python ../models/research/object_detection/model_main_tf2.py --model_dir={MY_MODEL_DIR} --pipeline_config_path={MY_MODEL_DIR + 'pipeline_adam.config'} --num_train_steps=5000

python ../models/research/object_detection/model_main_tf2.py --model_dir=./models/my_effiecientdet/ --pipeline_config_path=./models/my_effiecientdet/pipeline_adam.config --num_train_steps=5000


## Load model from ckpt

In [7]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [11]:
configs = config_util.get_configs_from_pipeline_file(MY_MODEL_DIR + 'pipeline.config')
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(MY_MODEL_DIR + 'ckpt-2').expect_partial()

2022-06-18 15:07:58.527501: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-18 15:07:58.538691: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-18 15:07:58.538927: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-18 15:07:58.539737: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [12]:
@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

## Inference

In [13]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [14]:
category_index = label_map_util.create_category_index_from_labelmap('./annotations/label_map.pbtxt')
category_index

{1: {'id': 1, 'name': 'helmet'},
 2: {'id': 2, 'name': 'head'},
 3: {'id': 3, 'name': 'person'}}

In [20]:
image_np = np.array(cv2.imread(TEST_DIR + 'hard_hat_workers218.png'))
input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
detections['num_detections'] = num_detections

detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.5,
            agnostic_mode=False)

print(detections)

cv2.imshow('detection', image_np_with_detections)
cv2.waitKey(0)
cv2.destroyAllWindows()

# out_image = cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB)
# plt.imshow(out_image)
# plt.show()

{'detection_boxes': array([[0.6300813 , 0.16133125, 0.7292936 , 0.26054388],
       [0.68765867, 0.1914855 , 0.82796645, 0.26163924],
       [0.59406734, 0.16422115, 0.8746829 , 0.30452865],
       [0.70848674, 0.8464309 , 0.8852634 , 0.9348191 ],
       [0.5169654 , 0.18236834, 0.5611596 , 0.2707565 ],
       [0.5325904 , 0.18236834, 0.5767846 , 0.27075645],
       [0.8237217 , 0.81150615, 0.87940294, 0.9228685 ],
       [0.471431  , 0.6459868 , 0.55981934, 0.8227632 ],
       [0.13549337, 0.5013404 , 0.22388184, 0.5455346 ],
       [0.3439086 , 0.7227356 , 0.48421657, 0.7928894 ],
       [0.80468744, 0.78906226, 0.8671873 , 0.8515624 ],
       [0.82031244, 0.80468726, 0.8828123 , 0.8671874 ],
       [0.8229934 , 0.15759045, 0.9113817 , 0.20178458],
       [0.83593744, 0.78906226, 0.8984373 , 0.8515624 ],
       [0.81265867, 0.16023548, 0.95296645, 0.23038931],
       [0.8488313 , 0.89570624, 0.9480437 , 0.99491876],
       [0.9167433 , 0.17321546, 1.        , 0.2174096 ],
       [0.9